In [1]:
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import numpy as np

movies = pd.read_csv("movies.csv")
ratings = pd.read_csv("ratings.csv")

movies_users = ratings.pivot(index="movieId", columns="userId", values="rating").fillna(0)
mat_movies = csr_matrix(movies_users.values)

movie_id_to_index = {movie_id: idx for idx, movie_id in enumerate(movies_users.index)}
index_to_movie_id = {idx: movie_id for movie_id, idx in movie_id_to_index.items()}

model = NearestNeighbors(metric="cosine", algorithm="brute", n_neighbors=50)  
model.fit(mat_movies)

def recommender_multiple_by_id(movie_ids, data, n):
    indices = [movie_id_to_index[movie_id] for movie_id in movie_ids if movie_id in movie_id_to_index]
    
    if not indices:
        print("No valid movie IDs provided.")
        return
    
    print("\nSearching for recommendations based on the selected movie IDs...")
    for movie_id in movie_ids:
        if movie_id in movie_id_to_index:
            print(f"Movie Selected: {movies[movies['movieId'] == movie_id]['title'].values[0]} | Movie ID: {movie_id}")
        else:
            print(f"Movie ID {movie_id} not found in dataset.")
    
    mean_vector = np.asarray(data[indices].mean(axis=0)).reshape(1, -1)
    
    distance, rec_indices = model.kneighbors(mean_vector, n_neighbors=50)  
    
    rec_movie_ids = [index_to_movie_id[i] for i in rec_indices[0] if i not in indices]
    
    if len(rec_movie_ids) < n:
        all_movie_ids = set(index_to_movie_id.values())
        additional_recs = [mid for mid in all_movie_ids if mid not in rec_movie_ids and mid not in movie_ids]
        rec_movie_ids.extend(additional_recs[:n - len(rec_movie_ids)])
    
    rec_movie_ids = rec_movie_ids[:n]  
    
    print("\nRecommended Movies:")
    for movie_id in rec_movie_ids:
        print(movies[movies['movieId'] == movie_id]['title'].values[0])

input_movie_ids = [1, 2, 4896, 13, 313, 48, 239, 364, 551] 
recommender_multiple_by_id(input_movie_ids, mat_movies, 10)



Searching for recommendations based on the selected movie IDs...
Movie Selected: Toy Story (1995) | Movie ID: 1
Movie Selected: Jumanji (1995) | Movie ID: 2
Movie Selected: Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001) | Movie ID: 4896
Movie Selected: Balto (1995) | Movie ID: 13
Movie Selected: Swan Princess, The (1994) | Movie ID: 313
Movie Selected: Pocahontas (1995) | Movie ID: 48
Movie Selected: Goofy Movie, A (1995) | Movie ID: 239
Movie Selected: Lion King, The (1994) | Movie ID: 364
Movie Selected: Nightmare Before Christmas, The (1993) | Movie ID: 551

Recommended Movies:
Aladdin (1992)
Beauty and the Beast (1991)
Mrs. Doubtfire (1993)
Jurassic Park (1993)
Mask, The (1994)
Forrest Gump (1994)
Snow White and the Seven Dwarfs (1937)
Home Alone (1990)
Babe (1995)
Shrek (2001)
